<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_Final_Pytorch/Pytorch/Linear_Regression_Dataset_No_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score
from google.colab import drive

!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 6.23 s (started: 2023-12-15 22:44:48 +00:00)


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.54 s (started: 2023-12-15 22:44:54 +00:00)


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Data/Processed Combined Final Dataset.csv'

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,1,140,289,304.342082,1.0,1.0,1.0,1.0,1.0,...,304.342082,304.342082,304.342082,304.342082,5.0,5.0,5.0,5.0,1.0,0
1,49,0,160,180,425.888148,1.0,1.0,1.0,1.0,1.0,...,425.888148,425.888148,425.888148,425.888148,7.0,7.0,7.0,7.0,1.0,1
2,37,1,130,283,299.019353,1.0,1.0,1.0,1.0,1.0,...,299.019353,299.019353,299.019353,299.019353,5.0,5.0,5.0,5.0,1.0,0
3,48,0,138,214,375.511197,1.0,1.0,1.0,1.0,1.0,...,375.511197,375.511197,375.511197,375.511197,5.0,5.0,5.0,5.0,1.0,1
4,54,1,150,195,408.567916,1.0,1.0,1.0,1.0,1.0,...,408.567916,408.567916,408.567916,408.567916,5.0,5.0,5.0,5.0,1.0,0


time: 50.3 ms (started: 2023-12-15 22:44:56 +00:00)


In [ ]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:23].values
Y = heart_attack.iloc[:, 24].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val = scale.transform(X_val)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

X_train_t = torch.tensor(X_train, dtype=torch.float)
X_val_t = torch.tensor(X_val, dtype=torch.float)
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(X_train_t.dtype)
print(X_val_t.dtype)
print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
time: 12.7 ms (started: 2023-12-15 22:44:56 +00:00)


In [ ]:
def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 795 µs (started: 2023-12-15 22:44:56 +00:00)


In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_t,
    val_t_in = X_val_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 56.5286, 	Validation loss 50.2810
Epoch 2, 	Training loss 48.2433, 	Validation loss 43.0094
Epoch 3, 	Training loss 41.2371, 	Validation loss 36.8514
Epoch 500, 	Training loss 0.4458, 	Validation loss 0.4299
Epoch 1000, 	Training loss 0.2905, 	Validation loss 0.2841
Epoch 1500, 	Training loss 0.2498, 	Validation loss 0.2463
Epoch 2000, 	Training loss 0.2391, 	Validation loss 0.2367
Epoch 2500, 	Training loss 0.2362, 	Validation loss 0.2344
Epoch 3000, 	Training loss 0.2354, 	Validation loss 0.2338
Epoch 3500, 	Training loss 0.2352, 	Validation loss 0.2337
Epoch 4000, 	Training loss 0.2351, 	Validation loss 0.2337
Epoch 4500, 	Training loss 0.2351, 	Validation loss 0.2338
Epoch 5000, 	Training loss 0.2351, 	Validation loss 0.2338


tensor([0.0091, 0.0159, 0.0012, 0.3660], requires_grad=True)

time: 26.9 s (started: 2023-12-15 22:44:56 +00:00)


In [ ]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")

Training Accuracy: 0.0662
Training Precision: 0.0404
Training Recall: 0.9099
Training F1 Score: 0.0775

Validation Accuracy: 0.0541
Validation Precision: 0.0501
Validation Recall: 0.9089
Validation F1 Score: 0.0949
time: 76.9 ms (started: 2023-12-15 22:45:23 +00:00)


In [ ]:
#Sequential Model

model_seq = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048, 512),
            nn.Tanh(),
            nn.Linear(512,1)
            )
model_seq

Sequential(
  (0): Linear(in_features=23, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 123 ms (started: 2023-12-15 22:45:23 +00:00)


In [ ]:
# Checking the complexity of the model
sum([p.numel() for p in model_seq.parameters()])

3173377

time: 6.3 ms (started: 2023-12-15 22:45:23 +00:00)


In [ ]:
#Let's define our training loop that will be used

def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 771 µs (started: 2023-12-15 22:45:23 +00:00)


In [ ]:
#Let's try doing some training

optimizer_1 = optim.SGD(model_seq.parameters(), lr=1e-2)

training_loop(
    n_epochs = 151,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_t,
    x_val = X_val_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.3795, Validation loss: 0.3758
Epoch: 10, Training loss: 0.2574, Validation loss: 0.2543
Epoch: 20, Training loss: 0.2364, Validation loss: 0.2334
Epoch: 30, Training loss: 0.2332, Validation loss: 0.2302
Epoch: 40, Training loss: 0.2324, Validation loss: 0.2294
Epoch: 50, Training loss: 0.2321, Validation loss: 0.2291
Epoch: 60, Training loss: 0.2319, Validation loss: 0.2289
Epoch: 70, Training loss: 0.2317, Validation loss: 0.2287
Epoch: 80, Training loss: 0.2316, Validation loss: 0.2286
Epoch: 90, Training loss: 0.2315, Validation loss: 0.2285
Epoch: 100, Training loss: 0.2314, Validation loss: 0.2284
Epoch: 110, Training loss: 0.2314, Validation loss: 0.2283
Epoch: 120, Training loss: 0.2313, Validation loss: 0.2283
Epoch: 130, Training loss: 0.2313, Validation loss: 0.2283
Epoch: 140, Training loss: 0.2312, Validation loss: 0.2282
Epoch: 150, Training loss: 0.2312, Validation loss: 0.2282
time: 4min 17s (started: 2023-12-15 22:45:23 +00:00)


In [ ]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positive = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positive) if (true_positives + false_positive) > 0 else 0


time: 1.32 ms (started: 2023-12-15 22:49:40 +00:00)


In [ ]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_t)))

# Calculate accuracy, recall, precision, and F1 score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_val_t)))

# Calculate accuracy, recall, precision, and F1 score for the validation set
accuracy_val = calculate_accuracy(train_predictions, Y_val_t)
recall_val = calculate_recall(train_predictions, Y_val_t)
precision_val = calculate_precision(train_predictions, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)


# Print the metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.3825
Training Recall: 1.0000
Training Precision: 0.3825
Training F1 Score: 0.5533

Validation Accuracy: 0.3791
Validation Recall: 1.0000
Validation Precision: 0.3791
Validation F1 Score: 0.5497
time: 516 ms (started: 2023-12-15 22:49:40 +00:00)


In [ ]:
# Defining the fully connected neural network

model_fnn = nn.Sequential(
            nn.Linear(23, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 34.1 ms (started: 2023-12-15 22:49:41 +00:00)


In [ ]:
# Checking the complexity of the model
sum([p.numel() for p in model_fnn.parameters()])

4779649

time: 3.94 ms (started: 2023-12-15 22:49:41 +00:00)


In [ ]:
# Setting up the parameters used in the training loop

learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 151

# This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.682186
Epoch: 10, Loss: 0.656962
Epoch: 20, Loss: 0.656887
Epoch: 30, Loss: 0.656115
Epoch: 40, Loss: 0.657733
Epoch: 50, Loss: 0.657016
Epoch: 60, Loss: 0.655314
Epoch: 70, Loss: 0.655450
Epoch: 80, Loss: 0.656212
Epoch: 90, Loss: 0.655460
Epoch: 100, Loss: 0.655297
Epoch: 110, Loss: 0.654615
Epoch: 120, Loss: 0.655158
Epoch: 130, Loss: 0.654787
Epoch: 140, Loss: 0.655481
Epoch: 150, Loss: 0.653900
time: 5min 48s (started: 2023-12-15 22:49:41 +00:00)


In [ ]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.617503
Training Precision: 0.308752
Training Recall: 0.500000
Training F1 score: 0.381763
time: 751 ms (started: 2023-12-15 22:55:30 +00:00)


In [ ]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
pecall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
p1_score_train = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Validation Precision: %f" % precision_val)
print("Validation Recall: %f" % recall_val)
print("Validation F1 score: %f" % F1_score_train)

Validation Accuracy: 0.620931
Validation Precision: 0.310466
Validation Recall: 1.000000
Validation F1 score: 0.381763
time: 191 ms (started: 2023-12-15 22:55:30 +00:00)
